In [62]:
# Created by Alec Sblendorio 
# May 27, 2021
# Calculate Ionospheric Pierce Points (IPP) using Satellite and Reciever Data 
# 
import numpy as np 
from io import StringIO 
from scipy.fft import fft, fftfreq
from scipy import signal
import matplotlib.pyplot as plt
import math 
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from io import BytesIO
import time
import pandas as pd
import datetime as dt
#import pymap3d as pm
#import gzip
import shutil
import os

In [68]:
# Code from Leslie Lamarche courtesy of GitHub
# parse_sp3.py
#

# df = pd.read_csv('sio05760.sp3')

filename = 'igs21490.sp3'
sat_ephem = {}  #initialize dictionary 
time = []  

with open(filename,'r') as sp3file:
    lines = sp3file.readlines()
    num_sat = int(lines[2].split()[1])
    sat_lin = lines[2]
    sat_lin2 = lines[3]
    
    for s in range(32):
        #sat_ephem['{:02}'.format(s+1)] = []
        sat = '{:02}'.format(s+1)
        if (sat in sat_lin) or (sat in sat_lin2):
                sat_ephem[sat] = []
        print(sat_ephem.keys())

    for j in range(22, len(lines)-1, num_sat+1):  #starting at 22 up to 1271 incrementing by 13 
        t = lines[j][2:-6]
        time.append(dt.datetime.strptime(t[:-50], ' %Y  %m %d  %H  %M  %S.%f'))
        
        for i in range(num_sat):
            s = lines[j+i+1].split()            
            sat_ephem[s[0][2:]] = []
            
            sat_ephem[s[0][2:]].append([float(s[1]),float(s[2]),float(s[3])])
    
for sat in sat_ephem:
    sat_ephem[sat] = np.array(sat_ephem[sat]).T*1000.
   # print(sat_ephem.keys())

dict_keys(['01'])
dict_keys(['01', '02'])
dict_keys(['01', '02', '03'])
dict_keys(['01', '02', '03', '04'])
dict_keys(['01', '02', '03', '04', '05'])
dict_keys(['01', '02', '03', '04', '05', '06'])
dict_keys(['01', '02', '03', '04', '05', '06', '07'])
dict_keys(['01', '02', '03', '04', '05', '06', '07', '08'])
dict_keys(['01', '02', '03', '04', '05', '06', '07', '08', '09'])
dict_keys(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10'])
dict_keys(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11'])
dict_keys(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'])
dict_keys(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13'])
dict_keys(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14'])
dict_keys(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15'])
dict_keys(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15',

ValueError: time data '' does not match format ' %Y  %m %d  %H  %M  %S.%f'

In [67]:
filepath = '/Users/alecsblendorio/AnacondaProjects/IPP '
filename = 'igs21100.sp3.Z'
print(os.path.join(filepath,filename[:-2]))
with gzip.open(os.path.join(filepath,filename), 'rb') as f_in, open(os.path.join(filepath,filename[:-2]), 'wb') as f_out:
	shutil.copyfileobj(f_in, f_out)

/Users/alecsblendorio/AnacondaProjects/IPP /igs21100.sp3


BadGzipFile: Not a gzipped file (b'\x00\x00')

In [56]:
# sat_ephem.keys()
print(sat_ephem.keys())

dict_keys(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32'])


In [ ]:
def Ion = klobuchar(fi,lambda,elev,azimuth,tow,alfa,beta):
    c=2.99792458e8             # speed of light
    deg2semi=1/180           # degees to semicircles
    semi2rad=pi              # semicircles to radians
    deg2rad=pi/180            # degrees to radians
    a=azimuth*deg2rad          # azimuth in radians
    e=elev*deg2semi            # elevation angle in semicircles
    
    psi = 0.0137 / (e+0.11) - 0.022      # Earth Centered angle
    lat_i = fi*deg2semi + psi*cos(a)     # Subionospheric lat

    if (lat_i > 0.416)
        lat_i = 0.416
        elseif(lat_i < -0.416)
        lat_i = -0.416
                                    
long_i = lambda*deg2semi + (psi*sin(a)/cos(lat_i*semi2rad)) # Subionospheric long
lat_m = lat_i + 0.064*cos((long_i-1.617)*semi2rad) # Geomagnetic latitude

t = 4.32e4*long_i + tow
t = mod(t,86400.)          # Seconds of day

if (t > 86400.):
    t = t - 86400.

if (t < 0.):
    t = t + 86400.

sF = 1. + 16.*(0.53-e)^3             # Slant factor
                                      # Period of model
per = beta(1) + beta(2)*lat_m + beta(3)*lat_m^2 +beta(4)*lat_m^3

if (per < 72000.):
    per = 72000.

x = 2.*pi*(t-50400.)/per            # Phase of the model (Max at 14.00 = 50400 sec local time)                               

amp = alpha(1) + alpha(2)*lat_m + alpha(3)*lat_m^2 +alpha(4)*lat_m^3 # Amplitude of the model
if(amp < 0.):
    amp = 0.


In [ ]:
def getPP(satpos,sv,recpos,pph,err=1.0):
    """
    get az and el to the satellite and repeatedly increase the range,
    converting to LLA each time to check the altitude. Stop when all
    the altitudes are within err of pph. Inputs satellite position
    array in ECEF, satellite number, receiver position in ECEF, pierce point
    height in km and error in km if you want.
    """

    rlat,rlon,ralt = ecef2geodetic(recpos)
    sataz,satel,satr = ecef2aer(satpos[:,0],satpos[:,1],satpos[:,2],rlat,rlon,ralt)

    r=np.zeros(len(satr))
    pplat,pplon,ppalt = aer2geodetic(sataz,satel,r,rlat,rlon,ralt)
    mask = (ppalt/1000 - pph) < 0

    while np.sum(mask)>0:
        r[mask]+=100
        pplat,pplon,ppalt = aer2geodetic(sataz,satel,r*1000,rlat,rlon,ralt)
        mask = (ppalt/1000 - pph) < 0

    sRange = r - 100.0
    eRange = r
    count = 0
    while not np.all(abs(ppalt/1000-pph)<err):
        count +=1
        mRange = (sRange + eRange) / 2.0
        pplat,pplon,ppalt = aer2geodetic(sataz,satel,mRange*1000,rlat,rlon,ralt)
        mask = ppalt/1000>pph
        eRange[mask] = mRange[mask]
        sRange[~mask] = mRange[~mask]

        if(count>100):
            raise TypeError('going too long')
            break

    ppalt = pph*1000

    return pplat,pplon,ppalt